# XGBoost Kriging First Pass
## Predicting Residuals (Predicting ERROR)

In [ ]:
!python --version

Python 3.10.6


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys

Mounted at /content/gdrive


In [ ]:
!pip install xgboost

In [ ]:
#imports
print("Starting imports")
#basics
import time
print("Starting timer.")
startTime = time.time()

import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import seaborn as sns

#cleaning
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler

#metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
# from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score


#XGBoost Specific
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from numpy import absolute

#hyperparams
from sklearn.model_selection import GridSearchCV

#h5 conversion
import h5py

#import validation tests
from scipy.spatial.distance import euclidean, cityblock
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr
from sklearn.model_selection import KFold

print("Imports Complete")

Starting imports
Starting timer.
Imports Complete


In [ ]:
datetimeCurr = datetime.datetime.now().strftime("%y%M%d_%H%M%S")
print(f"Current Datetime File Association: {datetimeCurr}")

Current Datetime File Association: 234727_194751


## Get Data

In [ ]:
#paths
mainPath = '/content/gdrive/MyDrive/Work/Projects-Google-Drive/NSF BigData REU Site/Big Data REU Year 3/Big Data REU 2023 Teams/REU 2023 Team 1: Ice Bed Topography Prediction/Research/Yi_Work/Kriging/'
data_full_ = '/content/gdrive/MyDrive/Work/Projects-Google-Drive/NSF BigData REU Site/Big Data REU Year 3/Big Data REU 2023 Teams/REU 2023 Team 1: Ice Bed Topography Prediction/Research/Predicted and preprocessed data sets /kriging-transpose-fixed-merged.csv' #training and test combined
#data_1201_ = mainPath + '0_Data/d1201_vMag.csv'
validation1201_ = mainPath + '0_Data/bed_BedMachine.h5' #solutions
# saveMapTo_ = mainPath + "2_Results/"

In [ ]:
#read data in
# df_train = pd.read_csv(df_train_)
# df_test_x = pd.read_csv(df_test_)
# df_test_y = pd.read_csv(df_pred_)

print("Reading Data In_")
df_all = pd.read_csv(data_full_)

# df1201 = pd.read_csv(data_1201_)

# physSol = pd.DataFrame(np.array(h5py.File(validation1201_)['bed_BedMachine']))
print("Data read in completed.")

Reading Data In_
Data read in completed.


## Create functions

In [ ]:
print("Establishing RMSPE functions.")

Establishing RMSPE functions.


In [ ]:
def rmspe(y_true, y_pred):
    return np.sqrt(np.nanmean(np.square(((y_true - y_pred) / y_true))))*100

In [ ]:
def rmspe_1(y_true, y_pred):
    return np.sqrt(np.nanmean(np.square(y_true - y_pred) / y_true))*100

In [ ]:
print("Function established.")

Function established.


## Clean Data/Prepare Data

In [ ]:
def remove_outliers_iqr(data, threshold=1.5):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    return data[~((data < lower_bound) | (data > upper_bound))]


In [ ]:
#VIEWING KRIGING DATA & COLUMNS

print(df_all.columns)
print(df_all.shape)
print(df_all.head(4))

Index(['Unnamed: 0.1', 'Unnamed: 0', 'surf_x', 'surf_y', 'surf_vx', 'surf_vy',
       'surf_elv', 'surf_dhdt', 'surf_SMB', 'track_bed_x', 'track_bed_y',
       'v_mag', 'track_bed_target', 'residual', 'prediction'],
      dtype='object')
(632612, 15)
   Unnamed: 0.1  Unnamed: 0    surf_x     surf_y    surf_vx     surf_vy  \
0             0           0 -278075.0 -1860125.0  -28.47723   10.470569   
1             1           1 -278075.0 -1843625.0 -217.74167    6.631936   
2             2           2 -278075.0 -1840025.0 -229.13731  114.668870   
3             3           3 -278075.0 -1841075.0 -237.63937   75.172966   

    surf_elv  surf_dhdt  surf_SMB   track_bed_x  track_bed_y       v_mag  \
0  952.47925  -1.099075 -0.650526 -278074.96875 -1860189.125   30.341151   
1  971.19580  -1.255387 -0.645288 -278074.87500 -1843669.250  217.842644   
2  964.41690  -1.717028 -0.712596 -278074.81250 -1839996.875  256.228134   
3  975.31230  -1.582383 -0.673964 -278074.65625 -1841013.500  249.245

In [ ]:
df_all

,Unnamed: 0.1,Unnamed: 0,surf_x,surf_y,surf_vx,surf_vy,surf_elv,surf_dhdt,surf_SMB,track_bed_x,track_bed_y,v_mag,track_bed_target,residual,prediction
0,0,0,-278075.0,-1860125.0,-28.47723,10.470569,952.47925,-1.099075,-0.650526,-278074.96875,-1860189.125,30.341151,355.881287,10.949721,344.931566
1,1,1,-278075.0,-1843625.0,-217.74167,6.631936,971.19580,-1.255387,-0.645288,-278074.87500,-1843669.250,217.842644,414.425964,-2.565612,416.991576
2,2,2,-278075.0,-1840025.0,-229.13731,114.668870,964.41690,-1.717028,-0.712596,-278074.81250,-1839996.875,256.228134,61.308609,-1.836433,63.145042
3,3,3,-278075.0,-1841075.0,-237.63937,75.172966,975.31230,-1.582383,-0.673964,-278074.65625,-1841013.500,249.245752,84.890762,-1.996846,86.887608
4,4,4,-278075.0,-1850525.0,-220.97133,-63.448890,941.26100,-0.996348,-0.876423,-278074.53125,-1850554.500,229.900175,448.794556,-29.512292,478.306848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632607,4995,4995,-268475.0,-1741025.0,-112.39719,-38.885475,1421.68760,-0.398355,0.307456,-268540.59375,-1740988.875,118.933631,182.030151,-0.002782,182.032933
632608,4996,4996,-268475.0,-1849025.0,-207.27214,-24.911695,1138.69810,-0.837633,-0.489208,-268540.53125,-1848952.375,208.763820,223.175201,-13.740054,236.915256
632609,4997,4997,-268475.0,-1800875.0,-175.57402,-64.130360,1293.72910,-1.172230,0.057662,-268540.46875,-1800871.500,186.919607,64.462059,4.453112,60.008947
632610,4998,4998,-268475.0,-1790675.0,-127.60057,-25.510916,1277.05220,-0.928526,-0.263651,-268540.40625,-1790652.500,130.125756,277.882507,0.730252,277.152255


In [ ]:
# applying the method
count_nan = df_all['prediction'].isnull().sum()

# printing the number of values present
# in the column
print('Number of NaN values present: ' + str(count_nan))

Number of NaN values present: 0


## Summary of Dataset



In [ ]:
df_all.describe()

,Unnamed: 0.1,Unnamed: 0,surf_x,surf_y,surf_vx,surf_vy,surf_elv,surf_dhdt,surf_SMB,track_bed_x,track_bed_y,v_mag,track_bed_target,residual,prediction
count,632612.000000,632612.000000,632612.000000,6.326120e+05,632612.000000,632612.000000,632612.000000,632612.000000,632612.000000,632612.000000,6.326120e+05,632612.000000,632612.000000,632612.000000,632612.000000
mean,4075.317835,4075.317835,-230181.361403,-1.790726e+06,-117.265164,-7.480275,1641.108027,-0.450674,0.138632,-230181.426484,-1.790726e+06,119.302269,307.802733,-0.087696,307.890430
std,2739.254268,2739.254268,42870.752812,5.266727e+04,63.993356,26.298517,358.950109,0.455291,0.298989,42870.657630,5.266723e+04,66.036374,179.881091,188.947803,259.844534
min,0.000000,0.000000,-278075.000000,-1.877525e+06,-497.349520,-181.754910,896.689940,-2.645623,-1.094386,-278074.968750,-1.877525e+06,11.685980,-671.101746,-87117.814268,-52262.918124
25%,1817.000000,1817.000000,-262925.000000,-1.839875e+06,-140.406780,-16.124752,1393.432400,-0.667299,0.029755,-262853.898438,-1.839837e+06,82.282743,218.821369,-1.837201,218.414473
50%,3647.000000,3647.000000,-239825.000000,-1.795775e+06,-100.727190,-3.122487,1593.786000,-0.332278,0.272718,-239751.295007,-1.795756e+06,101.579563,323.722229,0.000161,323.669948
75%,6046.000000,6046.000000,-213875.000000,-1.743575e+06,-81.798416,5.193039,1847.247900,-0.075294,0.360216,-213885.110603,-1.743600e+06,142.571990,422.135132,1.945586,421.731267
max,9999.000000,9999.000000,-98075.000000,-1.697525e+06,-11.685960,154.847060,2512.847200,0.120497,0.397950,-98075.164062,-1.697525e+06,519.833177,1735.522583,52628.282504,87468.511167


## Sort Data by Residual





In [ ]:
df_all.sort_values('residual')

,Unnamed: 0.1,Unnamed: 0,surf_x,surf_y,surf_vx,surf_vy,surf_elv,surf_dhdt,surf_SMB,track_bed_x,track_bed_y,v_mag,track_bed_target,residual,prediction
381197,1197,1197,-220025.0,-1699925.0,-81.473305,0.442332,1861.3782,-0.005738,0.373302,-220043.197138,-1.699925e+06,81.474506,350.696899,-87117.814268,87468.511167
575185,185,185,-114425.0,-1748525.0,-37.548927,4.118352,2433.5535,0.001194,0.371716,-114401.875000,-1.748482e+06,37.774102,267.657867,-27160.417094,27428.074961
580336,336,336,-108575.0,-1746575.0,-35.367004,5.419391,2458.5554,0.005348,0.373884,-108605.375000,-1.746593e+06,35.779809,265.473053,-25240.123424,25505.596477
543024,3024,3024,-143975.0,-1758125.0,-48.541378,2.536458,2307.3410,-0.014738,0.369354,-143998.112332,-1.758107e+06,48.607602,365.482605,-25014.309738,25379.792343
542796,2796,2796,-144275.0,-1758125.0,-48.661636,2.746714,2305.4040,-0.014363,0.369229,-144240.422903,-1.758126e+06,48.739094,364.353027,-24827.990949,25192.343976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573643,3643,3643,-116075.0,-1748975.0,-38.572870,3.316565,2426.8127,0.000304,0.370884,-116146.554688,-1.749012e+06,38.715190,335.628082,23084.962469,-22749.334387
585124,124,124,-103325.0,-1744775.0,-33.920826,6.238794,2481.6184,0.011309,0.371232,-103331.554688,-1.744835e+06,34.489781,231.004272,24512.975473,-24281.971200
542332,2332,2332,-144725.0,-1758125.0,-48.852100,2.466927,2302.4160,-0.013801,0.369020,-144742.457848,-1.758142e+06,48.914348,361.334534,27410.245882,-27048.911349
556343,6343,6343,-129725.0,-1742975.0,-41.019375,2.435334,2364.2020,0.001340,0.368556,-129703.070312,-1.743029e+06,41.091605,293.273560,46051.451200,-45758.177641


##  Data Standardization and Train-Test Split

In [ ]:
print("Begin data cleaning.")

# df1201 = df1201.drop(columns = ['Unnamed: 0.1'])
# df_all = df_all.drop(columns = ['Unnamed: 0.1'])

#1201 drop location variables
# df1201_feats = df1201.drop(columns = ['surf_x', 'surf_y'])

#DROP OUTLIERS & DUPLICATES
# Filter outliers from the numeric columns
# numeric_data = df_all.select_dtypes(include=[np.number])
# data_no_outliers = numeric_data.apply(lambda x: remove_outliers_iqr(x), axis=0)

# # Concatenate the non-numeric columns to the filtered numeric data
# non_numeric_data = df_all.select_dtypes(exclude=[np.number])
# df_all = pd.concat([non_numeric_data, data_no_outliers], axis=1).dropna()

# print(df_all.isna().sum())
# print(df_all.shape)

#################

#df_all drop location variables
df_all_feats_target = df_all.drop(columns = ['surf_x', 'surf_y', 'track_bed_x', 'track_bed_y', 'track_bed_target'])

#1201 order to align with df_all
# df1201_feats_ordered = df1201_feats[['surf_vx', 'surf_vy', 'surf_elv', 'surf_dhdt', 'surf_SMB', 'v_mag']]

#set the feature variables to our independent characteristic variables
#feature_cols = ['surf_vx', 'surf_vy', 'surf_elv', 'surf_dhdt', 'surf_SMB','v_mag', 'prediction']
feature_cols = ['surf_vx', 'surf_vy', 'surf_elv', 'surf_dhdt', 'surf_SMB','v_mag']

#split into X and Y
X_given = df_all_feats_target[feature_cols]
Y_given = df_all_feats_target['residual']

#combine all known X and validation 1201 X for standardizing
# X_all = np.concatenate((X_given, df1201_feats_ordered))
X_all = X_given
#make y into a dataframe to be standardized
Y_all = pd.DataFrame(Y_given)

print("Data cleaned.\nScaling beginning.")

#standardize
#Not setting feature range, let it be automatically determined
scaler_X = StandardScaler()
scaler_Y = StandardScaler()

X_all_std = scaler_X.fit_transform(X_all)
Y_all_std = scaler_Y.fit_transform(Y_all)

#can alternatively use the MinMaxScaler
print("Scaling Complete.\nSplitting Data Beginning.")

#split of 1201 data from X_all_std
X_non1201 = X_all_std[0:df_all.shape[0],:]
# X_1201_data = X_all_std[632706:, :]

# #generate a randomseed for training and testing split
# generated = np.random.randint(0,1000,1)[0]
# print(f"Generated random split for train-test: {generated}")
# # #generated = #
generated = 168

#set the train-test split
#60-40 showed the most promising from previous research and additional testing
train_size_ = .6
#split training and test from df_all
x_train, x_test, y_train, y_test = train_test_split(X_non1201, Y_all_std, train_size = train_size_, test_size = 1-train_size_, random_state = generated)

#get validation data
val_split = .2 #can change as needed
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size = 1-val_split, test_size = val_split, random_state = generated)

print(f"Data train-split complete with: {train_size_ * 100}% training, {(1- train_size_) * 100}% testing, {val_split*100}% validation")

Begin data cleaning.
Data cleaned.
Scaling beginning.
Scaling Complete.
Splitting Data Beginning.
Data train-split complete with: 60.0% training, 40.0% testing, 20.0% validation


## Modeling with XGB

In [ ]:
depth_ = 20 #higher --> Lower RMSE; lower --> more extreme highs and lows
iters_ = 50 #higher is higher R^2 and more reflection on the detail
eta_ = .4 #higher is more overfitting

In [ ]:
#define
model_residuals = XGBRegressor(
    max_depth=depth_,
    n_estimators=iters_,
    # min_child_weight=0.25,
    subsample=0.8,
    eta=eta_,
    seed=generated)

In [ ]:
%%time
model_residuals.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False
    )

CPU times: user 1min 30s, sys: 198 ms, total: 1min 30s
Wall time: 1min 2s


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.4, eval_metric=None,
             feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=20,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=50, n_jobs=None,
             num_parallel_tree=None, predictor=None, ...)

## Metrics

In [ ]:
%%time
print("Model Prediction Beginning")
y_pred_test = model_residuals.predict(x_test)

print("Model predicted.\nTransform data back to original scale.")

#get the original scale for the predicted Y
test_predict_full_scale = scaler_Y.inverse_transform(y_pred_test.reshape(-1,1))
y_test_given_full_range = scaler_Y.inverse_transform(y_test.reshape(-1,1))

# Evaluate the model performance
print("*****")
print("Print testing stats statements.")

#LOSSES
print('RMSE:',np.sqrt(mean_squared_error(y_test_given_full_range, test_predict_full_scale)))
# print('RMSE Percentage:',rmspe(y_test_given_full_range, test_predict_full_scale))
# print('RMSE Percentage-1:',rmspe_1(y_test_given_full_range, test_predict_full_scale))
print('Mean Absolute Error:', mean_absolute_error(y_test_given_full_range, test_predict_full_scale))
# print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(y_test_given_full_range, test_predict_full_scale))
print('R^2 Score:', r2_score(y_test_given_full_range, test_predict_full_scale))

Model Prediction Beginning
Model predicted.
Transform data back to original scale.
*****
Print testing stats statements.
RMSE: 142.2038477544784
Mean Absolute Error: 8.76811531730618
R^2 Score: -0.0831514298484699
CPU times: user 2.23 s, sys: 7.79 ms, total: 2.24 s
Wall time: 1.38 s


## Model Evaluation

In [ ]:
%%time
# Evaluate the combined model on the non-bias training data

# Pass the latent samples through the decoder
y_pred_val = model_residuals.predict(x_val)

#rescale
val_predict_full_scale = scaler_Y.inverse_transform(y_pred_val.reshape(-1,1))
y_val_given_full_range = scaler_Y.inverse_transform(y_val.reshape(-1,1))

# Evaluate the model performance
print("Print validation stats statements.")

#LOSSES
print('RMSE:',np.sqrt(mean_squared_error(y_val_given_full_range, val_predict_full_scale)))
# print('RMSE Percentage:',rmspe(y_val_given_full_range, val_predict_full_scale))
print('Mean Absolute Error:', mean_absolute_error(y_val_given_full_range, val_predict_full_scale))
# print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(y_val_given_full_range, val_predict_full_scale))
print('R^2 Score:', r2_score(y_val_given_full_range, val_predict_full_scale))

Print validation stats statements.
RMSE: 304.67772293419904
Mean Absolute Error: 10.434736844593594
R^2 Score: -0.020966579324048107
CPU times: user 677 ms, sys: 3.79 ms, total: 681 ms
Wall time: 666 ms


In [ ]:
print(f"predicted mean: {val_predict_full_scale.mean()}")
print(f"kriging mean: {y_val_given_full_range.mean()}")

print(f"predicted max: {val_predict_full_scale.max()}")
print(f"kriging max: {y_val_given_full_range.max()}")

print(f"predicted min: {val_predict_full_scale.min()}")
print(f"kriging min: {y_val_given_full_range.min()}")

predicted mean: -0.2448124885559082
kriging mean: 0.7421606604982134
predicted max: 1967.3304443359375
kriging max: 52628.282503705006
predicted min: -3892.347412109375
kriging min: -27160.417093515403


In [ ]:
# # Create the KDE plots
# sns.kdeplot(val_predict_full_scale.flatten(), label='Predicted', color="blue")
# sns.kdeplot(y_val_given_full_range.flatten(), label='Actual', color="orange")

# # Get the data for the box and whisker plot
# data = [val_predict_full_scale.flatten(), y_val_given_full_range.flatten()]
# labels = ['Predicted', 'Actual']

# # Create the box and whisker plot
# plt.figure()
# sns.boxplot(data=data, palette=["blue", "orange"])
# plt.xlabel('Value')
# plt.ylabel('Density (Frequency of Prediction)')
# plt.title('Validation Data - Predicted vs Actual')
# plt.xticks(range(2), labels)
# plt.show()

In [ ]:
df_all

,Unnamed: 0.1,Unnamed: 0,surf_x,surf_y,surf_vx,surf_vy,surf_elv,surf_dhdt,surf_SMB,track_bed_x,track_bed_y,v_mag,track_bed_target,residual,prediction
0,0,0,-278075.0,-1860125.0,-28.47723,10.470569,952.47925,-1.099075,-0.650526,-278074.96875,-1860189.125,30.341151,355.881287,10.949721,344.931566
1,1,1,-278075.0,-1843625.0,-217.74167,6.631936,971.19580,-1.255387,-0.645288,-278074.87500,-1843669.250,217.842644,414.425964,-2.565612,416.991576
2,2,2,-278075.0,-1840025.0,-229.13731,114.668870,964.41690,-1.717028,-0.712596,-278074.81250,-1839996.875,256.228134,61.308609,-1.836433,63.145042
3,3,3,-278075.0,-1841075.0,-237.63937,75.172966,975.31230,-1.582383,-0.673964,-278074.65625,-1841013.500,249.245752,84.890762,-1.996846,86.887608
4,4,4,-278075.0,-1850525.0,-220.97133,-63.448890,941.26100,-0.996348,-0.876423,-278074.53125,-1850554.500,229.900175,448.794556,-29.512292,478.306848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632607,4995,4995,-268475.0,-1741025.0,-112.39719,-38.885475,1421.68760,-0.398355,0.307456,-268540.59375,-1740988.875,118.933631,182.030151,-0.002782,182.032933
632608,4996,4996,-268475.0,-1849025.0,-207.27214,-24.911695,1138.69810,-0.837633,-0.489208,-268540.53125,-1848952.375,208.763820,223.175201,-13.740054,236.915256
632609,4997,4997,-268475.0,-1800875.0,-175.57402,-64.130360,1293.72910,-1.172230,0.057662,-268540.46875,-1800871.500,186.919607,64.462059,4.453112,60.008947
632610,4998,4998,-268475.0,-1790675.0,-127.60057,-25.510916,1277.05220,-0.928526,-0.263651,-268540.40625,-1790652.500,130.125756,277.882507,0.730252,277.152255


## Prediction for all features

In [ ]:
getResidualsFor = df_all[feature_cols]
getResidualsFor['PredResiduals'] = model_residuals.predict(getResidualsFor)

<ipython-input-30-c52417ee8747>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  getResidualsFor['PredResiduals'] = model_residuals.predict(getResidualsFor)


## Create a dataset based on XGB prediction


In [ ]:
getResidualsFor

,surf_vx,surf_vy,surf_elv,surf_dhdt,surf_SMB,v_mag,PredResiduals
0,-28.47723,10.470569,952.47925,-1.099075,-0.650526,30.341151,0.013418
1,-217.74167,6.631936,971.19580,-1.255387,-0.645288,217.842644,0.013418
2,-229.13731,114.668870,964.41690,-1.717028,-0.712596,256.228134,0.013418
3,-237.63937,75.172966,975.31230,-1.582383,-0.673964,249.245752,0.013418
4,-220.97133,-63.448890,941.26100,-0.996348,-0.876423,229.900175,-2.525530
...,...,...,...,...,...,...,...
632607,-112.39719,-38.885475,1421.68760,-0.398355,0.307456,118.933631,-2.525530
632608,-207.27214,-24.911695,1138.69810,-0.837633,-0.489208,208.763820,-2.525530
632609,-175.57402,-64.130360,1293.72910,-1.172230,0.057662,186.919607,-2.525530
632610,-127.60057,-25.510916,1277.05220,-0.928526,-0.263651,130.125756,-2.525530


In [ ]:
df_all['residual']

0         10.949721
1         -2.565612
2         -1.836433
3         -1.996846
4        -29.512292
            ...    
632607    -0.002782
632608   -13.740054
632609     4.453112
632610     0.730252
632611    -0.121602
Name: residual, Length: 632612, dtype: float64

## Adding columns on new residual datasset

---



In [ ]:
getResidualsFor['residual'] = df_all['residual'].values
getResidualsFor['track_bed_target'] = df_all['track_bed_target'].values
getResidualsFor['kriging_prediction'] = df_all['prediction'].values
getResidualsFor

<ipython-input-33-b11e19509844>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  getResidualsFor['residual'] = df_all['residual'].values
<ipython-input-33-b11e19509844>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  getResidualsFor['track_bed_target'] = df_all['track_bed_target'].values
<ipython-input-33-b11e19509844>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

,surf_vx,surf_vy,surf_elv,surf_dhdt,surf_SMB,v_mag,PredResiduals,residual,track_bed_target,kriging_prediction
0,-28.47723,10.470569,952.47925,-1.099075,-0.650526,30.341151,0.013418,10.949721,355.881287,344.931566
1,-217.74167,6.631936,971.19580,-1.255387,-0.645288,217.842644,0.013418,-2.565612,414.425964,416.991576
2,-229.13731,114.668870,964.41690,-1.717028,-0.712596,256.228134,0.013418,-1.836433,61.308609,63.145042
3,-237.63937,75.172966,975.31230,-1.582383,-0.673964,249.245752,0.013418,-1.996846,84.890762,86.887608
4,-220.97133,-63.448890,941.26100,-0.996348,-0.876423,229.900175,-2.525530,-29.512292,448.794556,478.306848
...,...,...,...,...,...,...,...,...,...,...
632607,-112.39719,-38.885475,1421.68760,-0.398355,0.307456,118.933631,-2.525530,-0.002782,182.030151,182.032933
632608,-207.27214,-24.911695,1138.69810,-0.837633,-0.489208,208.763820,-2.525530,-13.740054,223.175201,236.915256
632609,-175.57402,-64.130360,1293.72910,-1.172230,0.057662,186.919607,-2.525530,4.453112,64.462059,60.008947
632610,-127.60057,-25.510916,1277.05220,-0.928526,-0.263651,130.125756,-2.525530,0.730252,277.882507,277.152255


In [ ]:
#combined = pd.merge(getResidualsFor, df_all, how='inner', left_on=['surf_vx', 'surf_vy', 'surf_elv', 'surf_dhdt', 'surf_SMB'], right_on = ['surf_vx', 'surf_vy', 'surf_elv', 'surf_dhdt', 'surf_SMB'])
#combined

## Combining kriging prediction and XGB prediction

In [ ]:
#change the sign
getResidualsFor['Krig-PredRes'] = getResidualsFor['kriging_prediction'] + getResidualsFor['PredResiduals']

<ipython-input-35-12194cedcdc5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  getResidualsFor['Krig-PredRes'] = getResidualsFor['kriging_prediction'] + getResidualsFor['PredResiduals']


In [ ]:
getResidualsFor

,surf_vx,surf_vy,surf_elv,surf_dhdt,surf_SMB,v_mag,PredResiduals,residual,track_bed_target,kriging_prediction,Krig-PredRes
0,-28.47723,10.470569,952.47925,-1.099075,-0.650526,30.341151,0.013418,10.949721,355.881287,344.931566,344.944984
1,-217.74167,6.631936,971.19580,-1.255387,-0.645288,217.842644,0.013418,-2.565612,414.425964,416.991576,417.004995
2,-229.13731,114.668870,964.41690,-1.717028,-0.712596,256.228134,0.013418,-1.836433,61.308609,63.145042,63.158460
3,-237.63937,75.172966,975.31230,-1.582383,-0.673964,249.245752,0.013418,-1.996846,84.890762,86.887608,86.901027
4,-220.97133,-63.448890,941.26100,-0.996348,-0.876423,229.900175,-2.525530,-29.512292,448.794556,478.306848,475.781318
...,...,...,...,...,...,...,...,...,...,...,...
632607,-112.39719,-38.885475,1421.68760,-0.398355,0.307456,118.933631,-2.525530,-0.002782,182.030151,182.032933,179.507403
632608,-207.27214,-24.911695,1138.69810,-0.837633,-0.489208,208.763820,-2.525530,-13.740054,223.175201,236.915256,234.389726
632609,-175.57402,-64.130360,1293.72910,-1.172230,0.057662,186.919607,-2.525530,4.453112,64.462059,60.008947,57.483417
632610,-127.60057,-25.510916,1277.05220,-0.928526,-0.263651,130.125756,-2.525530,0.730252,277.882507,277.152255,274.626725


## Metrics for combined residuals for all known features

In [ ]:
krigResCorrection = getResidualsFor['Krig-PredRes'].dropna()
# krigPred = getResidualsFor['track_bed_target_kriging_prediction'].dropna('Krig+PredRes')
target = df_all['track_bed_target'].dropna()

print("Kriging-Predicted Residuals vs. Target Track Bed for All Known Data")
#LOSSES
print('RMSE:',np.sqrt(mean_squared_error(krigResCorrection, target)))
# print('RMSE Percentage:',rmspe(y_val_given_full_range, val_predict_full_scale))
print('Mean Absolute Error:', mean_absolute_error(krigResCorrection, target))
# print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(y_val_given_full_range, val_predict_full_scale))
print('R^2 Score:', r2_score(krigResCorrection, target))
print("*****")
# print('RMSE:',np.sqrt(mean_squared_error(krigPred, target)))
# # print('RMSE Percentage:',rmspe(y_val_given_full_range, val_predict_full_scale))
# print('Mean Absolute Error:', mean_absolute_error(krigPred, target))
# # print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(y_val_given_full_range, val_predict_full_scale))
# print('R^2 Score:', r2_score(krigPred, target))

Kriging-Predicted Residuals vs. Target Track Bed for All Known Data
RMSE: 188.95764883484546
Mean Absolute Error: 8.305075459253572
R^2 Score: 0.4721017866935021
*****


## MAE for combined residuals

In [ ]:
absolute_error = pd.DataFrame (krigResCorrection - target)
absolute_error

,0
0,-10.936302
1,2.579030
2,1.849851
3,2.010264
4,26.986763
...,...
632607,-2.522748
632608,11.214525
632609,-6.978642
632610,-3.255782


In [ ]:
(krigResCorrection - target).mean()

-1.4441310921340094

## Metrics based on Track bed target

In [ ]:
#Generate test data performance statistics for kriging residual learning results
krigResPred = getResidualsFor['Krig-PredRes']
known = getResidualsFor['track_bed_target']


train_size_ = .6
#split training and test from df_all
known_train, known_test, phys_train, phys_test = train_test_split(known, krigResPred, train_size = train_size_, test_size = 1-train_size_, random_state = 168)


# Evaluate the model performance
print("Print kriging residual model stats statements.")


#LOSSES
print('RMSE:',np.sqrt(mean_squared_error(known_test, phys_test)))
print('Mean Absolute Error:', mean_absolute_error(known_test, phys_test))
print('R^2 Score:', r2_score(known_test, phys_test))

Print kriging residual model stats statements.
RMSE: 136.6504668025484
Mean Absolute Error: 8.121896008643077
R^2 Score: 0.4237823010034881


In [ ]:
endTime = time.time()
print(f"Total Time Taken: {endTime - startTime:.03f}ms")
print("XGBT4M Kriging COMPLETE")

Total Time Taken: 83.897ms
XGBT4M Kriging COMPLETE


In [ ]:
#Generate test data performance statistics for kriging results only

krigPred = df_all['prediction']
known = df_all['track_bed_target']


train_size_ = .6
#split training and test from df_all
known_train, known_test, phys_train, phys_test = train_test_split(known, krigPred, train_size = train_size_, test_size = 1-train_size_, random_state = 168)


# Evaluate the model performance
print("Print kriging residual model stats statements.")


#LOSSES
print('RMSE:',np.sqrt(mean_squared_error(known_test, phys_test)))
print('Mean Absolute Error:', mean_absolute_error(known_test, phys_test))
print('R^2 Score:', r2_score(known_test, phys_test))

Print kriging residual model stats statements.
RMSE: 136.63651693567897
Mean Absolute Error: 7.6173676566658575
R^2 Score: 0.42389994055631086
